In [1]:
# Import library to read urls
from urllib2 import urlopen
# Import library to parse html
from bs4 import BeautifulSoup
from pymongo import MongoClient
import pandas as pd
import numpy as np
import re
import time

In [2]:
client = MongoClient()
# Access/Initiate Database
db = client['rap_db']
# Access/Initiate Table
tab = db['lyrics']

In [3]:
artists = ['dmx','toohort','pablo','masterp','50cent','drake','bone','saltnpepa',\
           'juvenile','youngjeezy','wizkhalifa','lilb','lilwayne','guccimane','missy',\
           'west','rundmc','ugk','snoopdogg','jadakiss','scarface','icp','nickiminaj',\
          'three6mafia','getoboys','ti','camron','bizmarkie','icecube','nelly','game',\
          'lupefiasco','lilkim','jayz','clipse','icet','puffdaddy','royceda59','eminem',\
          'cypress','krsone','mosdef','rakim','tribecalledquest','tyga','fatjoe',\
          'publicenemy','kweli','brotherali','twista','llcoolj','mobbdeep','ludacris',\
           'gangstarr','goodiemob','techn9ne','busta','wale','delasoul','methodman',\
          'common','raekwon','xzibit','beastie','nas','outkast','e40','blackalicious',\
          'redman','ghostface','roots','wutang','rza','canibus','gza','aesoprock',\
          'jcole','kendricklamar']
# 19/50cent.html
# 19/2pac

In [17]:
#Gather all song links for particular artist

artist = '2pac'

if artist[0].isdigit():
    url = 'http://www.azlyrics.com/19/' + artist + '.html'
else:
    url = 'http://www.azlyrics.com/' + artist[0] + '/' + artist + '.html'
    
content = urlopen(url).read()
# Feed the html to a BeatifulSoup object
soup = BeautifulSoup(content)
els = soup.find(id='listAlbum')

#Do not include song lyrics from "non-rap" songs or repeats (e.g., remixes)
stop_strings = ['intro','skit','interlude','remix']

#Parse out urls
urls = [x['href'][2:] for x in els.find_all(target='_blank') if not any(substr in x.contents[0].lower() for substr in stop_strings)]

In [16]:
# Feed the html to a BeatifulSoup object
soup = BeautifulSoup(content)
rows = soup.find('div',{'class':'col-xs-12 col-lg-8 text-center'})
rows = rows.find('div',{'class':''})
text = rows.text
text

u'\n\r\n("Hard like an erection..." -- Ed O.G.)  Young black male!\r\n("Hard like an erection...")\r\n("...ain\'t shit to fool with" -- Ice Cube)\r\n("Hard like an erection...")  Young black male!\r\n("...ain\'t shit to fool with")\n[runs backwards] Young black male!\n\n[2Pac] Yes niggaz... yes niggaz... yes niggaz\n\r\nYoung black male!\r\n("Hard like an erection...")  Young black male!\r\n("Hard like an erection...") ("...ain\'t shit to fool with")\r\n("Hard like an erection...")  Young black male!\r\n("Hard like an erection...") ("...ain\'t shit to fool with")\n\n[2Pac] Go nigga, go!!\n\n[Verse One:]\n\r\nYoung black male\r\nI try to effect by kicking the facts\r\nand stacking much mail\r\nI\'m packing a gat cuz guys wanna jack\r\nand fuck goin to jail\r\nCuz I ain\'t equipped to stop how I look\r\nI don\'t sell ya-yo\r\nThey teachin a brother like I was in books\r\nFollow me into a flow\r\nI\'m sure you know, which way to go\r\nI\'m hittin em out of the dopes\r\nSo slip on the slop

In [32]:
for i, url in enumerate(urls):
    
    print i, url
    
    # Go to the link and get the html as a string
    content = urlopen('http://www.azlyrics.com/' + url).read()
    # Feed the html to a BeatifulSoup object
    soup = BeautifulSoup(content)

    # Extract the rows in the table
    rows = soup.find('div',{'class':'col-xs-12 col-lg-8 text-center'})
    rows = rows.find('div',{'class':''})
    text = rows.text

    time.sleep(5)
    time.sleep(np.random.rand()*10)
    
    if i % 5 == 0:
        time.sleep(5)
        time.sleep(np.random.rand()*10)     
    
    lyrics = ''
    for t in a:
        if len(t) > 100:
            lyrics = lyrics + t.strip().encode('ascii','ignore')
    tab.insert_one({'artist':artist, 'url':url, 'lyrics':lyrics})

In [ ]:
#Export to JSON

# import pymongo
# from bson.json_util import dumps
# import json
# import pickle

# # connection = pymongo.Connection("localhost", 27017)
# # db = connection.mydocs

# def get():
#     cursor = tab.find()
#     return dumps(cursor)
# raps = json.loads(get())

# import cPickle as pickle
# with open("rap_data.json", 'w') as f:
#     pickle.dump(raps, f)